# HW3: Where is the Röstigraben?

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests as rq

## Data Wrangling

In [ ]:
# Defining NaN values:
na = ['NPO (Biblioth., Museen, Verwalt.)',# "Not precise enough"
       'Weitere Institute',                # Other institutes 
       'Firmen/Privatwirtschaft',          # "Not precise enough"
       'Nicht zuteilbar - NA',             # Not classifiable
       'data not included in P3']          # Missing value

df = pd.read_csv('data/P3_GrantExport.csv', delimiter=';', index_col=0, na_values=na,
                 parse_dates=["Start Date", "End Date"])
cantons = pd.read_csv('data/cantons_codes.csv', header=None, names=['Name'])

In [ ]:
no_amount = df[['Approved Amount']].isnull().mean()[0]
print('{:d}% of \'Approved Amount\' is NaN.'.format(int(100*no_amount)))

In [ ]:
# Splitting university name and abbreviation:
universities = df['University'].str.split(' - ', expand=True).rename(columns={0:'Name', 1:'Abbr'})
universities.dropna(axis=0,inplace=True)
universities.drop_duplicates(inplace=True)
universities = universities.set_index('Name')
print('There are {} different universities.'.format(universities.count()[0]))

# Filling the canton from abbreviations and add a location column:
universities['Canton'] = universities['Abbr'].apply(lambda x: x if x in cantons.values else None)
missing = universities['Canton'].isnull().sum()
print('We are still missing {} universities.'.format(missing))

In [ ]:
universities.head()

In [ ]:
# Using Google to retrieve missing canton and all locations:
# pip install python-google-places
from googleplaces import GooglePlaces, types, lang
from keys import GOOGLE_KEY
google_places = GooglePlaces(GOOGLE_KEY)

def search_google(place):
    place.get_details()
    location = place.geo_location
    for address in place.details['address_components']:
        canton = address['short_name']
        if canton in cantons.values:
            return canton, location
    return None, location

locations = {}
try:
    locations = pd.read_json('data/unis.json').to_dict()
except:
    for name in universities.index:
        r = google_places.text_search(query=name+' Schweiz')
        if r and r.places:
            canton, location = search_google(r.places[0]) # 1st result from query
            if not universities.at[name, 'Canton']:
                iversities.at[name, 'Canton'] = canton
            locations.update({name:{'Canton':canton,'Location':location}})
    
    manual = { 
    'Eidg. Material und Prüfungsanstalt':{'Canton':'ZH','Location':{'lat':47.429403 ,'lng':8.614036}},
    'Physikal.-Meteorolog. Observatorium Davos':{'Canton':'GR', 'Location':{'lat':46.814241, 'lng':9.844508}},
    'Forschungsinstitut für Opthalmologie':{'Canton':'VS', 'Location':{'lat':46.233131,  'lng':7.383104}},
    'Swiss Institute of Bioinformatics':{'Canton':'VD', 'Location':{'lat':46.519433, 'lng':6.574533}},
    'Pädag. Hochschule Tessin (Teilschule SUPSI)':{'Canton':'TI', 'Location':{'lat':46.023528, 'lng':8.917150}},
    'Staatsunabh. Theologische Hochschule Basel':{'Canton':'BL', 'Location':{'lat':47.577821, 'lng':7.650187}} 
    }
    unis = pd.DataFrame(dict(locations, **manual))
    unis.to_json('data/unis.json')
    locations = unis.to_dict()
print('Loaded {} universities.'.format(len(locations)))

In [ ]:
# Look at universities on a map
import folium
map1 = folium.Map(location=[46.798333, 8.231944],zoom_start=8)

for uni,v in locations.items():
    l = v['Location']
    c = v['Canton'] if v['Canton'] else ''
    p = uni+' - '+c
    folium.Marker([l['lat'],l['lng']],popup=p).add_to(map1)

map1.save('swiss_universities.html')

In [ ]:
map1